### Bibliotecas

In [116]:
import random
import math
from haversine import haversine

### Função de Avaliação (objetivo) ou (fitness)

In [117]:
# Função para calcular a distância entre duas cidades
def calcular_tamanho_rota(rotas, matriz_distancias):
    distancia_total = 0
    for i in range(len(rotas) - 1):
        cidade_a = rotas[i]
        cidade_b = rotas[i + 1]
        distancia_total += matriz_distancias[cidade_a][cidade_b]
    cidade_a = rotas[-1]
    cidade_b = rotas[0]
    distancia_total += matriz_distancias[cidade_a][cidade_b]
    return distancia_total

### Criar uma população inicial

In [118]:
# Função para gerar uma população inicial de rotas
def gerar_populacao_inicial(tamanho_populacao, n_cidades):
    populacao = []
    for _ in range(tamanho_populacao):
        rota = list(range(n_cidades))
        random.shuffle(rota)
        populacao.append(rota)
    return populacao

### Criar uma função de seleção

In [119]:
# Função para seleção de pais (roleta). Esta função retorna um pai
def selecao_por_roleta(populacao, matriz_distancias):
    fitness = []
    for rota in populacao:
        fitness.append(1 / calcular_tamanho_rota(rota, matriz_distancias))
    total_fitness = sum(fitness)
    probabilidades = [f / total_fitness for f in fitness]
    return populacao[random.choices(range(len(populacao)), probabilidades)[0]]

### Criar uma função de cruzamento

In [120]:
# Função para cruzamento de duas rotas (crossover) e retorna um filho
def cruzamento(rota1, rota2):
    start, end = sorted(random.sample(range(len(rota1)), 2))
    filho = [-1] * len(rota1)
    filho[start:end] = rota1[start:end]
    for i in range(len(rota2)):
        if rota2[i] not in filho:
            for j in range(len(filho)):
                if filho[j] == -1:
                    filho[j] = rota2[i]
                    break
    return filho

### Criar uma função de mutação

In [121]:
# Função de mutação (troca de dois pontos)
def mutacao(rota, taxa_mutacao):
    if random.random() < taxa_mutacao:
        i, j = random.sample(range(len(rota)), 2)
        rota[i], rota[j] = rota[j], rota[i]
    return rota

### Loop principal

In [122]:
# Função para o algoritmo genético
def algoritmo_genetico(matriz_distancias, tamanho_populacao=100, geracoes=100, taxa_mutacao=0.01):
    n_cidades = len(matriz_distancias)
    populacao = gerar_populacao_inicial(tamanho_populacao, n_cidades)
    melhor_rota = None
    melhor_distancia = float('inf')
    geracao_melhor_rota = -1

    for geracao in range(geracoes):
        nova_populacao = []
        for _ in range(len(populacao) // 2):
            pai1 = selecao_por_roleta(populacao, matriz_distancias)
            pai2 = selecao_por_roleta(populacao, matriz_distancias)
            filho1 = cruzamento(pai1, pai2)
            filho2 = cruzamento(pai2, pai1)
            nova_populacao.append(mutacao(filho1, taxa_mutacao))
            nova_populacao.append(mutacao(filho2, taxa_mutacao))
        
        populacao = nova_populacao

        # Verifica o melhor individuo da população
        for rota in populacao:
            distancia = calcular_tamanho_rota(rota, matriz_distancias)
            if distancia < melhor_distancia:
                melhor_distancia = distancia
                melhor_rota = rota
                geracao_melhor_rota = geracao

    return melhor_rota, melhor_distancia, geracao_melhor_rota

In [178]:
matriz_distancias = [
    [0,  10, 15, 20, 25, 30],
    [10, 0,  35, 25, 30, 5],
    [15, 35, 0,  30, 15, 20],
    [20, 25, 30, 0,  10, 15],
    [25, 30, 15, 10, 0,  10],
    [30, 5,  20, 15, 10, 0]
]

tamanho_populacao = 10
geracoes = 100
taxa_mutacao = 0.05

melhor_rota, melhor_distancia, geracao_melhor_rota = algoritmo_genetico(matriz_distancias, tamanho_populacao, geracoes, taxa_mutacao)

matriz_distancias
print(f"Melhor rota: {melhor_rota}")
print(f"Distância Total: {melhor_distancia}")
print(f"Geração que obteve a melhor solução: {geracao_melhor_rota}")

Melhor rota: [3, 4, 2, 0, 1, 5]
Distância Total: 70
Geração que obteve a melhor solução: 26


In [165]:
locations = {
    'ana_clara': (-19.738784523377877, -47.92374443626213),
    'arthur_ramos': (-19.751327, -47.921663),
    'artur_campos': (-19.7353056, -47.8839722),
    'barbara_cobo': (-19.7796514, -47.9128640),
    'bruno_antonio': (-19.781608483506588, -47.941363879846016),
    'davi_chaves': (-19.751448, -47.990355),
    'heitor_melo': (-19.7430957, -47.8985734),
    'joice_cristina': (-19.7403451, -47.9251389),
    'luan_flor': (-19.78146, -47.91386),
    'lucas_dayrell': (-19.74213256406242,  -47.93437766952385),
    'luiz_folador': (-19.73502, -47.93592),
    'maria_luisa': (-19.735548, -47.885493),
    'patrick_machado': (-19.752190, -47.925118),
    'rogerio_coutinho': (-19.740709, -47.945302),
    'tayna_cardoso': (-19.78146, -47.9325588),
    'iftm': (-19.718660337225800, -47.95776847526162)
}

matrix = []
names = list(locations.keys())

for name1 in names:
    row = []
    for name2 in names:
        distance = haversine(locations[name1], locations[name2])
        row.append(distance)
    matrix.append(row)


tamanho_populacao = 10
geracoes = 100
taxa_mutacao = 0.05

melhor_rota, melhor_distancia, geracao_melhor_rota = algoritmo_genetico(matrix, tamanho_populacao, geracoes, taxa_mutacao)


print("Distance Matrix (km):")
print("        " + "  ".join(f"{n[:8]:>8}" for n in names))
for name, row in zip(names, matrix):
    print(f"{name[:8]:>8}  " + "  ".join(f"{d:8.4f}" for d in row))


print(f"\n")
print(f"Melhor rota: {melhor_rota}")
print(f"Distância Total: {melhor_distancia}")
print(f"Geração que obteve a melhor solução: {geracao_melhor_rota}")

Distance Matrix (km):
        ana_clar  arthur_r  artur_ca  barbara_  bruno_an  davi_cha  heitor_m  joice_cr  luan_flo  lucas_da  luiz_fol  maria_lu  patrick_  rogerio_  tayna_ca      iftm
ana_clar    0.0000    1.4116    4.1806    4.6847    5.1063    7.1121    2.6777    0.2267    4.8567    1.1735    1.3413    4.0196    1.4975    2.2664    4.8341    4.2059
arthur_r    1.4116    0.0000    4.3283    3.2814    3.9481    7.1889    2.5840    1.2742    3.4487    1.6781    2.3483    4.1723    0.3741    2.7413    3.5393    5.2416
artur_ca    4.1806    4.3283    0.0000    5.7842    7.9108   11.2776    1.7566    4.3449    6.0101    5.3299    5.4372    0.1614    4.6977    6.4470    7.2244    7.9429
barbara_    4.6847    3.2814    5.7842    0.0000    2.9900    8.6943    4.3312    4.5555    0.2265    4.7406    5.5182    5.6793    3.3119    5.5022    2.0705    8.2510
bruno_an    5.1063    3.9481    7.9108    2.9900    0.0000    6.1261    6.1960    4.8924    2.8779    4.4500    5.2116    7.7728    3.6